In [ ]:
from utils import parse_wiki_article, process_articles_directory

ImportError: cannot import name 'process_articles_directory' from 'src.utils.ArticlesParser' (/Users/alexislimozin/Documents/ada-2024-project-collectifmetisser/src/utils/ArticlesParser.py)